In [1]:
import numpy as np
import scipy.stats as si
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

In [2]:
def black_scholes(S, K, T, r, sigma, q, option_type):
    if S <= 0 or K <= 0 or T <= 0 or sigma <= 0:
        raise ValueError("Ensure that S, K, T, and sigma must be positive values.")
    if option_type not in ['call', 'put']:
        raise ValueError("option_type must be 'call' or 'put'")

    d1 = (np.log(S / K)) + (r - q + 0.5 * sigma ** 2) * T
    d1 /= (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        price = S * np.exp(-q * T) * si.norm.cdf(d1) - K * np.exp(-r * T) * si.norm.cdf(d2)
    else:
        price = K * np.exp(-r * T) * si.norm.cdf(-d2) - S * np.exp(-q * T) * si.norm.cdf(-d1)

    return price

S_widget = widgets.FloatText(min=0.1, step=1)
K_widget = widgets.FloatText(min=0.1, step=1)
T_widget = widgets.FloatText(min=0.01, step=0.1)
sigma_widget = widgets.FloatText(min=0.01, max=5.0, step=0.01)
r_widget = widgets.FloatText(min=0.0, max=0.5, step=0.01)
q_widget = widgets.FloatText(min=0.0, max=0.5, step=0.01)

row1 = widgets.HBox([widgets.Label('Spot Price:'), S_widget])
row2 = widgets.HBox([widgets.Label('Strike Price:'), K_widget])
row3 = widgets.HBox([widgets.Label('Time to Expiry (Y):'), T_widget])
row4 = widgets.HBox([widgets.Label('Volatility:'), sigma_widget])
row5 = widgets.HBox([widgets.Label('Risk-free Rate:'), r_widget])
row6 = widgets.HBox([widgets.Label('Dividend Yield:'), q_widget])

input_box = widgets.VBox([row1, row2, row3, row4, row5, row6])

output = widgets.Output()
button = widgets.Button(description='Calculate Option Price')

def on_button_clicked(b):
    with output:
        output.clear_output()
        try:
            S = S_widget.value
            K = K_widget.value
            T = T_widget.value
            r = r_widget.value
            sigma = sigma_widget.value
            q = q_widget.value

            call_price = black_scholes(S, K, T, r, sigma, q, 'call')
            put_price = black_scholes(S, K, T, r, sigma, q, 'put')

            print(f"European Call Option Price: ${call_price:.2f}")
            print(f"European Put Option Price: ${put_price:.2f}")

            plt.figure(figsize=(10, 6))
            S_values = np.linspace(0.5*S, 1.5*S, 100)
            call_prices = [black_scholes(s, K, T, r, sigma, q, 'call') for s in S_values]
            put_prices = [black_scholes(s, K, T, r, sigma, q, 'put') for s in S_values]

            plt.plot(S_values, call_prices, label='Call Option', color='blue')
            plt.plot(S_values, put_prices, label='Put Option', color='red')
            plt.axvline(x=K, color='black', linestyle='--', label='Strike Price')
            plt.xlabel('Stock Price ($)')
            plt.ylabel('Option Price ($)')
            plt.title('Black-Scholes Option Pricing')
            plt.legend()
            plt.grid(True)
            plt.show()

        except Exception as e:
            print(f"Error: {str(e)}")

button.on_click(on_button_clicked)

display(input_box)
display(button)
display(output)

Button(description='Calculate Option Price', style=ButtonStyle())

Output()